##Question 1:

To start with, we'll need to compute the number of rentals per station per day. Use pandas to do that.


In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import datetime
from pandas import DataFrame

In [2]:
stations = pd.read_table('stations.tsv')

In [3]:
usage = pd.read_table('usage_2012.tsv')

In [4]:
usage['time_start'] = pd.to_datetime(usage['time_start'])

In [42]:
usage_per_day = usage.set_index('time_start').groupby(pd.TimeGrouper('D')).size()

In [43]:
usage_per_station = usage[['station_start']].groupby('station_start').size()

In [33]:
usage_per_station_per_day = usage.groupby(['date', 'station_start']).size()

In [36]:
usage_per_station_per_day

date      station_start                                   
1/1/2012  10th & Monroe St NE                                  10
          10th & U St NW                                       58
          10th St & Constitution Ave NW                        54
          11th & H St NE                                       20
          11th & Kenyon St NW                                  58
          12th & Army Navy Dr                                  12
          12th & Newton St NE                                   2
          13th & D St NE                                       46
          13th & H St NE                                       50
          13th St & New York Ave NW                            10
          14th & D St NW / Ronald Reagan Building              26
          14th & D St SE                                       32
          14th & G St NW                                       16
          14th & Harvard St NW                                 80
          14th & 

In [27]:
def to_date(x):
    return str(x.month) + '/' + str(x.day) + '/' + str(x.year)

In [28]:
usage['date'] = usage['time_start'].apply(to_date)

In [29]:
usage['date'].first

<bound method Series.first of 0            1/1/2012
1            1/1/2012
2            1/1/2012
3            1/1/2012
4            1/1/2012
5            1/1/2012
6            1/1/2012
7            1/1/2012
8            1/1/2012
9            1/1/2012
10           1/1/2012
11           1/1/2012
12           1/1/2012
13           1/1/2012
14           1/1/2012
15           1/1/2012
16           1/1/2012
17           1/1/2012
18           1/1/2012
19           1/1/2012
20           1/1/2012
21           1/1/2012
22           1/1/2012
23           1/1/2012
24           1/1/2012
25           1/1/2012
26           1/1/2012
27           1/1/2012
28           1/1/2012
29           1/1/2012
              ...    
2412494    12/31/2012
2412495    12/31/2012
2412496    12/31/2012
2412497    12/31/2012
2412498    12/31/2012
2412499    12/31/2012
2412500    12/31/2012
2412501    12/31/2012
2412502    12/31/2012
2412503    12/31/2012
2412504    12/31/2012
2412505    12/31/2012
2412506    12/31/2012
24

In [37]:
usage_per_station_per_day = usage.groupby(['date', 'station_start']).size()

In [38]:
usage_per_station_per_day

date      station_start                                   
1/1/2012  10th & Monroe St NE                                  10
          10th & U St NW                                       58
          10th St & Constitution Ave NW                        54
          11th & H St NE                                       20
          11th & Kenyon St NW                                  58
          12th & Army Navy Dr                                  12
          12th & Newton St NE                                   2
          13th & D St NE                                       46
          13th & H St NE                                       50
          13th St & New York Ave NW                            10
          14th & D St NW / Ronald Reagan Building              26
          14th & D St SE                                       32
          14th & G St NW                                       16
          14th & Harvard St NW                                 80
          14th & 

we have grouped our data by day then station and the size gives us the users in that day

2a. Our stations data has a huge number of quantitative attributes: fast_food, parking, restaurant, etc... Some of them are encoded as 0 or 1 (for absence or presence), others represent counts. To start with, run a simple linear regression where the input (x) variables are all the various station attributes and the output (y) variable is the average number of rentals per day.

In [42]:
rentals_per_station = usage[['station_start']].groupby('station_start').size()

In [43]:
def get_average(x):
    return x/366
daily_rentals = rentals_per_station.apply(get_average)

In [44]:
daily_rentals.head()

station_start
10th & E St NW                   17
10th & Monroe St NE              10
10th & U St NW                   71
10th St & Constitution Ave NW    55
11th & H St NE                   35
dtype: int64

In [45]:
from sklearn.linear_model import LinearRegression

In [47]:
stations_columns = list(stations.columns.values)

In [71]:
stations['cool_attributes'] = stations[stations_columns[7:]].sum(axis=1)

In [52]:
y = daily_rentals

In [101]:
x = stations[list(stations.columns[7:])]

In [105]:
model = LinearRegression()

In [106]:
model.fit(x,y)

ValueError: Found arrays with inconsistent numbers of samples: [185 347]

In [107]:
len(y)

185

In [108]:
len(x)

347